In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import thư viện cần thiết
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn import tree
from scipy.stats import f_oneway, chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [ ]:
# Đọc dữ liệu
booking_data_path = '/kaggle/input/airlinecustomer-2033210533/Passanger_booking_data.csv'
review_data_path = '/kaggle/input/airlinecustomer-2033210533/AirlineScrappedReview.csv'
booking_data = pd.read_csv(booking_data_path)
review_data = pd.read_csv(review_data_path)

# Phân Tích ANOVA

In [ ]:
# Phân loại 'review_title' là 'Good' hoặc 'Bad' dựa trên `rating` (sử dụng .loc để tránh cảnh báo)
review_data.loc[:, 'Review_Title_Class'] = review_data['Rating'].apply(lambda x: 'Good' if x >= 6 else 'Bad')

# Xây dựng mô hình OLS (Ordinary Least Squares) cho ANOVA
model = ols('Rating ~ Review_Title_Class', data=review_data).fit()

# Thực hiện phân tích ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)

# Hiển thị kết quả
print(anova_table)

In [ ]:
# Phân loại 'Review_Content' dựa trên 'Rating'
review_data['Review_Content_Class'] = review_data['Rating'].apply(lambda x: 'Good' if x > 5 else 'Bad')

# Vẽ boxplot để so sánh Rating theo Review_Content_Class
plt.figure(figsize=(10, 6))
sns.boxplot(x='Review_Content_Class', y='Rating', data=review_data, palette=["red", "green"])
plt.title('Boxplot of Ratings by Review Content')
plt.xlabel('Review Content')
plt.ylabel('Rating')
plt.show()

# Loại bỏ các giá trị NaN trong cột Rating
review_data = review_data.dropna(subset=['Rating'])

# Thực hiện kiểm định ANOVA giữa hai nhóm 'Good' và 'Bad'
good_ratings = review_data[review_data['Review_Content_Class'] == 'Good']['Rating']
bad_ratings = review_data[review_data['Review_Content_Class'] == 'Bad']['Rating']

f_stat, p_value = f_oneway(good_ratings, bad_ratings)

# In kết quả kiểm định
print("Kết quả kiểm định ANOVA:")
print("Giá trị F:", f_stat)
print("Giá trị p:", p_value)

# Chọn mức ý nghĩa (ví dụ: alpha = 0.05)
alpha = 0.05

if p_value < alpha:
    print("Bác bỏ giả thuyết H0: Có sự khác biệt thống kê có ý nghĩa giữa hai nhóm đánh giá.")
else:
    print("Không đủ bằng chứng để bác bỏ giả thuyết H0: Không có sự khác biệt thống kê có ý nghĩa giữa hai nhóm đánh giá.")

In [ ]:
# Xem các lớp trong cột phân loại
print(review_data['Class'].value_counts())

# Kiểm tra số lượng thuộc tính cho mỗi lớp
# Ví dụ, giả sử bạn muốn kiểm tra các thuộc tính trong cột 'class'
class_attributes = review_data.groupby('Class').size()

print(class_attributes)

In [ ]:
# Danh sách các lớp cần giữ lại
classes_to_keep = [
    'Economy Class', 'Business', 'Business Class', 'Premium Economy', 'First Class'
]

# Lọc dữ liệu để chỉ giữ lại các lớp cần thiết
filtered_df = review_data[review_data['Class'].isin(classes_to_keep)]

# Vẽ biểu đồ boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(x='Class', y='Rating', data=filtered_df)
plt.xticks(rotation=45)
plt.title('Boxplot of Rating by Class')
plt.show()

In [ ]:
# Danh sách các lớp cần giữ lại
classes_to_keep = [
    'Economy Class', 'Business', 'Business Class', 'Premium Economy', 'First Class'
]

# Lọc dữ liệu để chỉ giữ lại các lớp cần thiết
filtered_df = review_data[review_data['Class'].isin(classes_to_keep)]

# Tạo danh sách các nhóm để ANOVA
groups = [filtered_df[filtered_df['Class'] == c]['Rating'] for c in classes_to_keep]

# Thực hiện phân tích ANOVA
f_statistic, p_value = f_oneway(*groups)

# Hiển thị kết quả
print(f'F-statistic: {f_statistic:.4f}')
print(f'p-value: {p_value:.4f}')

# Kiểm tra kết quả
alpha = 0.05
if p_value < alpha:
    print('Có sự khác biệt đáng kể về điểm số đánh giá giữa các lớp.')
else:
    print('Không có sự khác biệt đáng kể về điểm số đánh giá giữa các lớp.')

In [ ]:
# Lọc những đánh giá tốt (rating từ 6 đến 10)
good_ratings = review_data[(review_data['Rating'] >= 6) & (review_data['Rating'] <= 10)]

# Lọc các loại class mà bạn yêu cầu
filtered_classes = good_ratings[good_ratings['Class'].isin(['Economy Class', 'Business', 'Business Class', 'Premium Economy', 'First Class'])]

# Đếm số lượng rating tốt theo từng loại class
class_counts = filtered_classes['Class'].value_counts()

print(class_counts)

# HỒI QUY TUYẾN TÍNH

**Cho 2 biến định lượng**

In [ ]:
# Biến giải thích: purchase_lead (định lượng), Biến phản hồi: flight_duration (định lượng)
X_quant = booking_data[['purchase_lead']]  # Predictor: purchase_lead
y_quant = booking_data['flight_duration']  # Response: flight_duration

# Thêm hằng số vào biến giải thích
X_quant = sm.add_constant(X_quant)
model_quant = sm.OLS(y_quant, X_quant).fit()

In [ ]:
# Vẽ biểu đồ phân tán kèm đường hồi quy
plt.figure(figsize=(8, 6))
plt.scatter(X_quant['purchase_lead'], y_quant, color='blue', alpha=0.3, label='Dữ liệu thực tế')
plt.plot(X_quant['purchase_lead'], model_quant.predict(X_quant), color='red', label='Đường hồi quy')
plt.xlabel('Purchase Lead (Số ngày trước chuyến đi)')
plt.ylabel('Flight Duration (Thời gian bay)')
plt.title('Hồi quy tuyến tính giữa Purchase Lead và Flight Duration')
plt.legend()
plt.show()

In [ ]:
# Xây dựng mô hình hồi quy tuyến tính
print(f"Mô hình hồi quy tuyến tính cho 2 biến định lượng")

# Hiển thị kết quả của mô hình hồi quy
print(model_quant.summary())

**biến giải thích phân loại và biến phản hồi định lượng**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# One-hot encoding biến 'sales_channel' để dùng cho mô hình hồi quy
encoder = OneHotEncoder(drop='first', sparse=False)  # Drop một mức để tránh đa cộng tuyến
sales_channel_encoded = encoder.fit_transform(booking_data[['sales_channel']])

In [ ]:
# Chuyển các cột được mã hóa sang DataFrame
sales_channel_encoded_df = pd.DataFrame(sales_channel_encoded, columns=encoder.get_feature_names_out(['sales_channel']))

In [ ]:
# Gộp các cột đã mã hóa vào dữ liệu gốc
data_encoded = pd.concat([booking_data, sales_channel_encoded_df], axis=1)

In [ ]:
# Định nghĩa biến giải thích (sales_channel) và biến phản hồi (flight_hour)
X_cat = sales_channel_encoded_df  # Sử dụng mã hóa one-hot cho 'sales_channel'
y_cat = booking_data['flight_hour']  # Biến phản hồi: flight_hour

In [ ]:
# Thêm hằng số cho statsmodels
X_cat_const = sm.add_constant(X_cat)

In [ ]:
# Tạo mô hình hồi quy tuyến tính
model_cat = sm.OLS(y_cat, X_cat_const).fit()

print(f"Mô hình hồi quy tuyến tính với biến giải thích phân loại và biến phản hồi định lượng")
# Hiển thị kết quả của mô hình hồi quy
print(model_cat.summary())

In [ ]:
# Tính toán giá trị trung bình và độ lệch chuẩn của flight_hour cho từng kênh bán hàng
mean_std_flight_hour = booking_data.groupby('sales_channel')['flight_hour'].agg(['mean', 'std']).reset_index()
mean_std_flight_hour.columns = ['Kênh bán hàng', 'Giá trị Trung bình', 'Độ lệch chuẩn']
# Hiển thị kết quả
print(mean_std_flight_hour)

In [ ]:
# Tính toán giá trị trung bình và độ lệch chuẩn của flight_hour cho từng kênh bán hàng
mean_std_flight_hour = booking_data.groupby('sales_channel')['flight_hour'].agg(['mean', 'std']).reset_index()
mean_std_flight_hour.columns = ['Kênh bán hàng', 'Giá trị Trung bình', 'Độ lệch chuẩn']

# Vẽ biểu đồ cột
plt.figure(figsize=(10, 6))
colors = ['#66cdaa', '#ffa07a']  # Màu sắc cho các cột
sns.barplot(x='Kênh bán hàng', y='Giá trị Trung bình', data=mean_std_flight_hour, palette=colors)
plt.xlabel('Sales Channel (Kênh bán hàng)')
plt.ylabel('Mean Flight Hour (Giờ bay trung bình)')
plt.title('Giờ bay trung bình theo từng kênh bán hàng')
plt.show()

# Phân tích Chi-square

In [ ]:
# Tạo bảng tần số chéo giữa 'sales_channel' và 'booking_complete'
cross_tab = pd.crosstab(booking_data['sales_channel'], booking_data['booking_complete'], normalize='index')

# Chỉ lấy tỷ lệ hoàn tất đặt chỗ (Booking Complete = 1)
complete_rates = cross_tab[1]  # Chỉ lấy cột có giá trị là "Hoàn tất"

# Vẽ biểu đồ cột chỉ cho tỷ lệ hoàn tất
plt.figure(figsize=(8, 6))
ax = sns.barplot(x=complete_rates.index, y=complete_rates.values, palette="Set2")

# Thêm giá trị chú thích trên các cột
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='baseline', fontsize=12, color='black', xytext=(0, 1), textcoords='offset points')

# Thêm tiêu đề và nhãn
plt.title("Tỷ lệ Đặt Chỗ Hoàn Tất theo Sales Channel", fontsize=16)
plt.xlabel("Sales Channel", fontsize=12)
plt.ylabel("Tỷ lệ Đặt Chỗ Hoàn Tất (%)", fontsize=12)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

# Tính toán Chi-square, p-value và bảng giá trị kỳ vọng
chi2, p, dof, expected = chi2_contingency(pd.crosstab(booking_data['sales_channel'], booking_data['booking_complete']))

# Tạo bảng giá trị kỳ vọng từ kết quả
expected_df = pd.DataFrame(expected, index=['Internet', 'Mobile'], columns=[0, 1])

# Tạo bảng tần số chéo
cross_tab_full = pd.crosstab(booking_data['sales_channel'], booking_data['booking_complete'])

# Tô màu cho bảng giá trị kỳ vọng
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

# Hiển thị bảng giá trị kỳ vọng với màu
print("Bảng giá trị kỳ vọng:")
display(expected_df.style.apply(highlight_max))

# Hiển thị bảng tần số chéo với màu
def color_cells(val):
    if val == 0:
        return 'background-color: lightcoral'
    else:
        return 'background-color: lightblue'

styled_cross_tab = cross_tab_full.style.applymap(color_cells)

print("Bảng tần số chéo giữa sales_channel và booking_complete:")
display(styled_cross_tab)

# In kết quả
print(f"Chi-square: {chi2:.4f}")
print(f"p-value: {p:.4e}")
print(f"Degrees of freedom: {dof}")

# CÂY QUYẾT ĐỊNH

In [ ]:
# Kiểm tra các giá trị thiếu
print("Số lượng giá trị thiếu trong mỗi cột:\n", booking_data.isnull().sum())

# Mã hóa các biến phân loại
label_encoders = {}
for column in booking_data.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    booking_data[column] = label_encoders[column].fit_transform(booking_data[column].astype(str))

# Chọn đặc trưng (features) và mục tiêu (target)
X = booking_data.drop('booking_complete', axis=1)  # Cột 'booking_complete' là mục tiêu
y = booking_data['booking_complete']  # Cột 'booking_complete' là mục tiêu

# Chia dữ liệu thành bộ huấn luyện và bộ kiểm tra (70% huấn luyện, 30% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Xây dựng mô hình cây quyết định
clf = DecisionTreeClassifier(random_state=42, max_depth=4)  # Giới hạn độ sâu của cây
clf.fit(X_train, y_train)

# Dự đoán trên bộ kiểm tra
y_pred = clf.predict(X_test)

# Đánh giá mô hình
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Hiển thị ma trận nhầm lẫn (Confusion Matrix)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


# Biểu đồ phân phối của 'booking_complete' (giá trị thực tế)
plt.figure(figsize=(6, 4))
sns.countplot(x='booking_complete', data=booking_data, palette='viridis')
plt.title('Distribution of Booking Completion (Actual)')
plt.xlabel('Booking Completed (Actual)')
plt.ylabel('Count')
plt.show()

# Biểu đồ phân phối giữa giá trị thực tế và giá trị dự đoán (so sánh riêng biệt trong các subplot)
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Biểu đồ phân phối thực tế
sns.countplot(x='booking_complete', data=pd.DataFrame({'booking_complete': y_test}), palette='Blues', ax=ax[0])
ax[0].set_title('Actual Booking Completion')
ax[0].set_xlabel('Booking Completed (Actual)')
ax[0].set_ylabel('Count')

# Vẽ cây quyết định với matplotlib
plt.figure(figsize=(14, 8))  # Điều chỉnh kích thước của đồ thị
tree.plot_tree(clf, filled=True, feature_names=X.columns, class_names=['Not Completed', 'Completed'],
               fontsize=10, rounded=True, proportion=True, precision=2)
plt.title('Decision Tree for Booking Completion')
plt.show()

# Biểu đồ phân phối dự đoán
sns.countplot(x='booking_complete', data=pd.DataFrame({'booking_complete': y_test, 'predictions': y_pred}), hue='predictions', palette='coolwarm', ax=ax[1])
ax[1].set_title('Predicted Booking Completion')
ax[1].set_xlabel('Booking Completed (Actual)')
ax[1].set_ylabel('Count')
ax[1].legend(title='Predicted', labels=['Not Completed', 'Completed'])

plt.tight_layout()  # Đảm bảo các biểu đồ không bị chồng lên nhau
plt.show()




# HÒI QUY LOGISTIC


In [ ]:
# Mô hình 1: Hồi quy logistic giữa 1 biến giải thích nhị phân và 1 biến phản hồi nhị phân
# Chọn biến giải thích nhị phân (wants_extra_baggage) và biến phản hồi nhị phân (booking_complete)
X_task1 = booking_data[['wants_extra_baggage']]  # Biến giải thích
y_task1 = booking_data['booking_complete']  # Biến phản hồi

# Chia dữ liệu thành tập huấn luyện và kiểm tra (70% huấn luyện, 30% kiểm tra)
X_train_task1, X_test_task1, y_train_task1, y_test_task1 = train_test_split(X_task1, y_task1, test_size=0.3, random_state=42)

# Khởi tạo và huấn luyện mô hình hồi quy logistic
log_reg_task1 = LogisticRegression()
log_reg_task1.fit(X_train_task1, y_train_task1)

# Dự đoán trên tập kiểm tra
y_pred_task1 = log_reg_task1.predict(X_test_task1)
y_prob_task1 = log_reg_task1.predict_proba(X_test_task1)[:, 1]  # Xác suất dự đoán

# Đánh giá qua độ chính xác, ma trận nhầm lẫn và báo cáo phân loại
accuracy_task1 = accuracy_score(y_test_task1, y_pred_task1)
conf_matrix_task1 = confusion_matrix(y_test_task1, y_pred_task1)
class_report_task1 = classification_report(y_test_task1, y_pred_task1)
print(f"Độ chính xác: {accuracy_task1}")
print(f"Ma trận nhầm lẫn:\n{conf_matrix_task1}")
print(f"Báo cáo phân loại:\n{class_report_task1}")


# Vẽ đồ thị ROC Curve
fpr_task1, tpr_task1, _ = roc_curve(y_test_task1, y_prob_task1)
roc_auc_task1 = roc_auc_score(y_test_task1, y_prob_task1)
plt.figure(figsize=(10, 5))
plt.plot(fpr_task1, tpr_task1, label='AUC  = %0.2f' % roc_auc_task1)
plt.plot([0, 1], [0, 1], linestyle='--', color='r')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Mô hình 1')
plt.legend(loc="lower right")
plt.show()



# Thêm cột intercept
booking_data['intercept'] = 1.0

# Định nghĩa biến giải thích và biến phản hồi cho statsmodels
X_sm = booking_data[['intercept', 'wants_extra_baggage']]
y_sm = booking_data['booking_complete']

# Áp dụng mô hình hồi quy logistic từ statsmodels
logit_model = sm.Logit(y_sm, X_sm)
result = logit_model.fit()

# In ra kết quả tóm tắt của mô hình
print(result.summary())

# Lấy hệ số của Intercept và biến wants_extra_baggage
intercept = result.params['intercept']
wants_extra_baggage_coef = result.params['wants_extra_baggage']

print(f'Intercept: {intercept}')
print(f'wants_extra_baggage: {wants_extra_baggage_coef}')

In [ ]:
# Mô hình 2: Hồi quy logistic giữa 3 biến giải thích (nhị phân, định lượng, phân loại) và biến phản hồi nhị phân
# Xử lý biến phân loại 'trip_type' bằng phương pháp One-Hot Encoding
X_task2 = pd.get_dummies(booking_data[['wants_extra_baggage', 'purchase_lead', 'trip_type']], drop_first=True)

# Biến phản hồi nhị phân
y_task2 = booking_data['booking_complete'].astype(int)

# Chia dữ liệu thành tập huấn luyện và kiểm tra (70% huấn luyện, 30% kiểm tra)
X_train_task2, X_test_task2, y_train_task2, y_test_task2 = train_test_split(X_task2, y_task2, test_size=0.3, random_state=42)

# Khởi tạo và huấn luyện mô hình hồi quy logistic cho Task 2
log_reg_task2 = LogisticRegression(max_iter=1000)
log_reg_task2.fit(X_train_task2, y_train_task2)

# Dự đoán trên tập kiểm tra
y_pred_task2 = log_reg_task2.predict(X_test_task2)
y_prob_task2 = log_reg_task2.predict_proba(X_test_task2)[:, 1]  # Xác suất dự đoán

# Đánh giá mô hình qua độ chính xác, ma trận nhầm lẫn và báo cáo phân loại
accuracy_task2 = accuracy_score(y_test_task2, y_pred_task2)
conf_matrix_task2 = confusion_matrix(y_test_task2, y_pred_task2)
class_report_task2 = classification_report(y_test_task2, y_pred_task2)

print(f"Độ chính xác của Mô hình 2: {accuracy_task2}")
print(f"Ma trận nhầm lẫn:\n{conf_matrix_task2}")
print(f"Báo cáo phân loại:\n{class_report_task2}")


# Vẽ đồ thị ROC Curve
fpr_task2, tpr_task2, _ = roc_curve(y_test_task2, y_prob_task2)
roc_auc_task2 = roc_auc_score(y_test_task2, y_prob_task2)
plt.figure(figsize=(10, 5))
plt.plot(fpr_task2, tpr_task2, label='AUC  = %0.2f' % roc_auc_task2)
plt.plot([0, 1], [0, 1], linestyle='--', color='r')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Mô hình 2')
plt.legend(loc="lower right")
plt.show()

# Thêm cột intercept cho statsmodels
X_sm_task2 = sm.add_constant(X_task2)

# Chuyển đổi dữ liệu thành định dạng số liệu phù hợp
X_sm_task2 = X_sm_task2.astype(float)
y_sm_task2 = y_task2.astype(float)

# Áp dụng mô hình hồi quy logistic từ statsmodels
logit_model_task2 = sm.Logit(y_sm_task2, X_sm_task2)
result_task2 = logit_model_task2.fit()

# In ra kết quả tóm tắt của mô hình
print(result_task2.summary())

# Lấy hệ số của Intercept và các biến giải thích
intercept_task2 = result_task2.params['const']
wants_extra_baggage_coef_task2 = result_task2.params['wants_extra_baggage']
purchase_lead_coef_task2 = result_task2.params['purchase_lead']
trip_type_coef_task2 = result_task2.params[X_sm_task2.columns[-1]]  # Cột cuối cùng của One-Hot Encoding

print(f'Intercept: {intercept_task2}')
print(f'wants_extra_baggage: {wants_extra_baggage_coef_task2}')
print(f'purchase_lead: {purchase_lead_coef_task2}')
print(f'trip_type: {trip_type_coef_task2}')